In [1]:
options(warn=-1)

In [36]:
library(aod)
library(caret)
library(dplyr)
library(forcats)
library(ggplot2)
library(MASS)
library(matrixStats)
library(MLmetrics)
library(skimr)
library(stats)
library(stringdist)
library(tidyr)
library(vcd)

In [3]:
size <- 10000

### I. Data preprocessing

In [4]:
data <- read.table('adult.data', sep=',')

In [5]:
colnames(data) <- scan('adult.names', what=character(), sep='\n')

In [6]:
data$`native-country`[as.character(data$`native-country`) == ' ?'] <- NA
data$workclass[as.character(data$workclass) == ' ?'] <- NA
data$occupation[as.character(data$occupation) == ' ?'] <- NA

In [7]:
head(data)

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [8]:
data$income <- factor(data$income, order=T, levels=c(' <=50K', ' >50K'))

Remove redundant features.

In [9]:
data$`education-num` <- NULL
data$`relationship` <- NULL
data$occupation <- NULL
data$fnlwgt <- NULL

Reducing number of levels of some factors.

In [10]:
data <- mutate(data, workclass = fct_collapse(workclass, Gov = c(' State-gov', ' Federal-gov', ' Local-gov')))
data <- mutate(data, workclass = fct_collapse(workclass, Self_employed = c(' Self-emp-not-inc', ' Self-emp-inc')))
data <- mutate(data, workclass = fct_collapse(workclass, Not_working = c(' Without-Pay', ' Never-worked')))

In [11]:
data <- mutate(data, education = fct_collapse(education, School = c(' 10th', ' 11th', ' 12th', ' 1st-4th', 
                                                                    ' 5th-6th', ' 7th-8th', ' 9th', ' Preschool', 
                                                                    ' HS-grad')))
data <- mutate(data, education = fct_collapse(education, Associates = c(' Assoc-acdm', ' Assoc-voc')))

data$education <- factor(data$education, order=T, levels=c('School', ' Prof-school', 'Associates', ' Some-college', 
                                                           ' Bachelors', ' Masters', ' Doctorate'))

In [12]:
europe <- c(' England', ' France', ' Germany', ' Greece', ' Holand-Netherlands', ' Hungary', ' Ireland', ' Italy', ' Poland', ' Portugal', ' Scotland', ' Yugoslavia')
asia <- c(' Cambodia', ' China', ' Hong', ' India', ' Japan', ' Laos', ' Philippines', ' Taiwan', ' Thailand', ' Vietnam')
central_america <- c(' Cuba', ' Dominican-Republic', ' El-Salvador', ' Guatemala', ' Haiti', ' Honduras', ' Jamaica', ' Mexico', ' Nicaragua', ' Puerto-Rico', ' Trinadad&Tobago')
south_america <- c(' Columbia', ' Ecuador', ' Peru')
usa <- c(' United-States', ' Outlying-US(Guam-USVI-etc)')

data <- mutate(data, `native-country` = fct_collapse(`native-country`, Europe = europe))
data <- mutate(data, `native-country` = fct_collapse(`native-country`, Asia = asia))
data <- mutate(data, `native-country` = fct_collapse(`native-country`, Central_America = central_america))
data <- mutate(data, `native-country` = fct_collapse(`native-country`, South_America = south_america))
data <- mutate(data, `native-country` = fct_collapse(`native-country`, USA = usa))

In [13]:
data <- mutate(data, `marital-status` = fct_collapse(`marital-status`, Married = c(' Married-AF-spouse', 
                                                                                   ' Married-civ-spouse')))
data <- mutate(data, `marital-status` = fct_collapse(`marital-status`, Not_married = c(' Married-spouse-absent', 
                                                                                       ' Separated', ' Divorced')))

Categorize some features.

In [14]:
age_factors <- function(x) {
    if (x <= 25) {
        return('Young')
    } else if (x <= 45) {
        return('Middle-aged')
    } else if (x <= 65) {
        return('Senior')
    } else {
        return('Old')
    }
}

data$age <- sapply(data$age, age_factors)

In [15]:
job_factors <- function(x) {
    if (x <= 25) {
        return('Part-time')
    } else if (x <= 40) {
        return('Full-time')
    } else {
        return('Over-time')
    }
}

data$`hours-per-week` <- sapply(data$`hours-per-week`, job_factors)

In [16]:
capital_factors <- function(x) {
    if (x == 0) {
        return('None')
    } else if (x <= med) {
        return('Low')
    } else {
        return('High')
    }
}

med <- median(data$`capital-gain`[data$`capital-gain` != 0])
data$`capital-gain` <- sapply(data$`capital-gain`, capital_factors)

med <- median(data$`capital-loss`[data$`capital-loss` != 0])
data$`capital-loss` <- sapply(data$`capital-loss`, capital_factors)

In [17]:
nominal_features <- c('workclass', 'marital-status', 'age', 'hours-per-week', 'capital-gain', 'capital-loss', 
                      'native-country')

data[nominal_features] <- lapply(data[nominal_features], as.factor)

Handle imbalanced target classes by removing a part of samples.

In [18]:
small_income <- data[data$income == ' <=50K',]
large_income <- data[data$income == ' >50K',]
sample <- small_income[sample(nrow(small_income), size), ]

In [19]:
data <- rbind(sample, large_income)
data <- slice(data, sample(1:n()))
rownames(data) <- NULL

In [20]:
skim_to_list(data)

variable,missing,complete,n,n_unique,top_counts,ordered
age,0,17841,17841,4,"Mid: 9372, Sen: 5269, You: 2617, Old: 583",FALSE
capital-gain,0,17841,17841,3,"Non: 15746, Hig: 1143, Low: 952, NA: 0",FALSE
capital-loss,0,17841,17841,3,"Non: 16758, Hig: 596, Low: 487, NA: 0",FALSE
education,0,17841,17841,7,"Sch: 7045, So: 3790, Ba: 3500, Ass: 1396",TRUE
hours-per-week,0,17841,17841,3,"Ful: 9914, Ove: 6219, Par: 1708, NA: 0",FALSE
income,0,17841,17841,2,"<=: 10000, >5: 7841, NA: 0",TRUE
marital-status,0,17841,17841,4,"Mar: 10061, Ne: 4568, Not: 2747, Wi: 465",FALSE
native-country,322,17519,17841,8,"USA: 16097, Cen: 543, Asi: 381, NA: 322",FALSE
race,0,17841,17841,5,"Wh: 15523, Bl: 1468, As: 593, Am: 136",FALSE
sex,0,17841,17841,2,"Ma: 12795, Fe: 5046, NA: 0",FALSE


Dealing with missing values by finding the closest sample.

In [21]:
workclass_na <- dplyr::filter(data, is.na(workclass) & !is.na(`native-country`))
country_na <- dplyr::filter(data, is.na(`native-country`) & !is.na(workclass))

In [22]:
data <- data[complete.cases(data), ]

In [23]:
workclass_na$workclass <- 0
country_na$`native-country` <- 0

In [24]:
closest <- function(x, column) {
    return(data[which.min(Reduce(`+`, Map(stringdist, data, x, method='jaccard'))),][, column])
}

workclass_na$workclass <- apply(workclass_na, 1, closest, 'workclass')
country_na$`native-country` <- apply(country_na, 1, closest, 'native-country')

In [25]:
data <- rbind(data, workclass_na, country_na)
data <- slice(data, sample(1:n()))
rownames(data) <- NULL

In [26]:
# mode <- function(x) {
#     ux <- unique(x)
#     ux[which.max(tabulate(match(x, ux)))]
# }

# data <- replace_na(data=data,
#                    replace=list(workclass = mode(data$workclass),
#                                 `native-country` = mode(data$`native-country`)))

In [27]:
head(data)

age,workclass,education,marital-status,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
Middle-aged,Gov,Some-college,Married,White,Male,None,None,Full-time,USA,>50K
Middle-aged,Gov,Some-college,Married,White,Male,None,None,Over-time,USA,>50K
Middle-aged,Private,School,Married,White,Male,None,None,Full-time,Central_America,<=50K
Senior,Private,School,Not_married,White,Female,None,None,Full-time,USA,<=50K
Senior,Private,Prof-school,Not_married,Asian-Pac-Islander,Female,None,None,Full-time,Asia,<=50K
Middle-aged,Gov,Associates,Not_married,Asian-Pac-Islander,Female,None,None,Full-time,USA,<=50K


In [28]:
skim_to_list(data)

variable,missing,complete,n,n_unique,top_counts,ordered
age,0,17828,17828,4,"Mid: 9365, Sen: 5266, You: 2615, Old: 582",FALSE
capital-gain,0,17828,17828,3,"Non: 15734, Hig: 1142, Low: 952, NA: 0",FALSE
capital-loss,0,17828,17828,3,"Non: 16745, Hig: 596, Low: 487, NA: 0",FALSE
education,0,17828,17828,7,"Sch: 7039, So: 3788, Ba: 3496, Ass: 1395",TRUE
hours-per-week,0,17828,17828,3,"Ful: 9908, Ove: 6215, Par: 1705, NA: 0",FALSE
income,0,17828,17828,2,"<=: 9991, >5: 7837, NA: 0",TRUE
marital-status,0,17828,17828,4,"Mar: 10055, Ne: 4566, Not: 2742, Wi: 465",FALSE
native-country,0,17828,17828,8,"USA: 16360, Cen: 548, Asi: 405, Eur: 301",FALSE
race,0,17828,17828,5,"Wh: 15514, Bl: 1467, As: 591, Am: 136",FALSE
sex,0,17828,17828,2,"Ma: 12789, Fe: 5039, NA: 0",FALSE


### II. Build the model

In [29]:
set.seed(666)
train_rows <- createDataPartition(data$income, p = 0.8, list = F)

In [30]:
train <- data[train_rows, ]
test <- data[-train_rows, ]

In [31]:
model <- train(data = train, income ~ ., family = binomial(link = 'logit'), method = 'glm')
summary(model)


Call:
NULL

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.1454  -0.5688  -0.1237   0.6480   2.8983  

Coefficients: (1 not defined because of singularities)
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                           -7.643e+11  1.134e+12  -0.674 0.500322
ageOld                                -2.500e-01  1.454e-01  -1.720 0.085510
ageSenior                              4.505e-01  5.171e-02   8.711  < 2e-16
ageYoung                              -1.754e+00  1.380e-01 -12.713  < 2e-16
workclassGov                           7.643e+11  1.134e+12   0.674 0.500322
workclassNot_working                   7.643e+11  1.134e+12   0.674 0.500322
`workclass Private`                    7.643e+11  1.134e+12   0.674 0.500322
workclassSelf_employed                 7.643e+11  1.134e+12   0.674 0.500322
`workclass Without-pay`                7.643e+11  1.134e+12   0.674 0.500322
education.L                            1.619e+00

In [32]:
preds <- predict(model, test)

In [33]:
confusionMatrix(data = preds, reference = test$income)

Confusion Matrix and Statistics

          Reference
Prediction  <=50K  >50K
     <=50K   1628   318
     >50K     370  1249
                                          
               Accuracy : 0.807           
                 95% CI : (0.7937, 0.8199)
    No Information Rate : 0.5604          
    P-Value [Acc > NIR] : < 2e-16         
                                          
                  Kappa : 0.6097          
                                          
 Mcnemar's Test P-Value : 0.05185         
                                          
            Sensitivity : 0.8148          
            Specificity : 0.7971          
         Pos Pred Value : 0.8366          
         Neg Pred Value : 0.7715          
             Prevalence : 0.5604          
         Detection Rate : 0.4567          
   Detection Prevalence : 0.5459          
      Balanced Accuracy : 0.8059          
                                          
       'Positive' Class :  <=50K          
               

In [34]:
probs <- predict(model, newdata = test, type = 'prob')
test_set <- data.frame(' <=50K' = probs$` <=50K`, ' >50K' = probs$` >50K`, pred = preds, obs = test$income)

In [35]:
prSummary(test_set, lev = levels(test_set$obs))[2:4]

Precision    Recall         F 
0.8365879 0.8148148 0.8255578